# Simple HMM in NumPy or PyTorch

In [ ]:
import numpy as np

In [ ]:
import torch
device = torch.device("cuda:0")
#dtype = torch.float
dtype = torch.double
x = torch.rand(5, 3, device=device, dtype=dtype)
print(x)

In [ ]:
# my matrix class based on either NumPy or PyTorch
class m:
    pass

use_torch = True
#use_torch = False

if use_torch:
    def m_torch_array(inp):
        return torch.tensor(inp).float()
    m.array = m_torch_array
    
    m.zeros = torch.zeros
else:
    m.array = np.array
    m.zeros = np.zeros

In [ ]:
# transition matrix [from,to]
A = m.array([[0.7,0.3,0,0],
            [0,0.9,0.1,0],
            [0,0,0.8,0.2],
            [0,0,0,1]])

In [ ]:
# where we can be
x = m.array([[1,0,0,0]])

In [ ]:
# recording length in frames
tmax = 14000

In [ ]:
alpha_m = m.zeros((tmax,len(x.T))) # mantissa-like values, kept in reasonable range
alpha_exp = m.zeros(tmax)
exponent = 0
base = 1000 # base for separation to exp and m (somewhere around 5e-324 starts underflow)

In [ ]:
b = alpha_m + 0.1

In [ ]:
%%time
x_m = m.array([[1,0,0,0]]) # mantissa-like moderated value (kept in range)
for row in range(tmax):
    while x_m.max()<1/base: # renormalize and remember power of base used
        x_m *= base
        exponent -= 1
    alpha_exp[row] = exponent
    alpha_m[row] = x_m
    x_m = x_m@A*b[row]   # FINETUNE WHICH row's b IS USED (ALSO FOR beta)

In [ ]:
alpha_m

In [ ]:
#At = np.transpose(A)
At = A.T


In [ ]:
beta_m = m.zeros((tmax,len(x.T)))
beta_exp = m.zeros(tmax)
exponent = 0

In [ ]:
x_m = m.array([[0,0,0,1]])
for row in range(tmax-1,0-1,-1):
    beta_m[row] = x_m
    beta_exp[row] = exponent
    x_m = x_m@At*b[row]
    
    while x_m.max()<1/base: # renormalize and remember power of base used
        x_m *= base
        exponent -= 1

In [ ]:
beta_m

In [ ]:
L_m = alpha_m*beta_m # this is .*

In [ ]:
L_m

In [ ]:
L_exp = alpha_exp+beta_exp
L_exp -= L_exp.max()

In [ ]:
L_exp.min()

In [ ]:
# re-normalize L

In [ ]:
L_m.max()

In [ ]:
for row in range(tmax):
    while L_exp[row]<0: # renormalize and remember power of base used
        L_m[row] *= 1/base
        L_exp[row] += 1

In [ ]:
L_exp.min(), L_exp.max()

In [ ]:
list(L_m.sum(axis=1))